In [1]:
import warnings
import matplotlib.animation as animation 
from IPython import display 
import numpy as np
import h5py
import matplotlib.pyplot as plt

In [2]:
#from phi.flow import *  # minimal dependencies
# from phi.torch.flow import *
# from phi.tf.flow import *

from phi.jax.flow import *

In [3]:
Res       = 200   # Resolution
Walls = Obstacle(union(
    [Box(x=(20, 60), y=(50, 51)), #bottom wall
    Box(x=(80, 100), y=(70, 71)), #top bar
    Box(x=(20, 21), y=(50, 65)),
    Box(x=(45, 46), y=(50, 65)),
    Box(x=(30, 31), y=(80, 100)),
    ]
))


In [ ]:
hf = h5py.File("Smoke-Flow-Data-temp.h5", "w")


for source in np.linspace(10,90,9):

    print('source at ', source)


    @jit_compile  # Only for PyTorch, TensorFlow and Jax
    def step(v, s, p, dt=1.):
        s = advect.mac_cormack(s, v, dt) + inflow
        buoyancy = resample(s * (0, 0.1), to=v)
        v = advect.semi_lagrangian(v, v, dt) + buoyancy * dt
        v, p = fluid.make_incompressible(v, Walls, Solve('auto', 1e-4, 0, x0=p))
        return v, s, p
    
    
    velocity = StaggeredGrid(0, x=Res, y=Res, bounds=Box(x=100, y=100))  # or CenteredGrid(...)
    smoke    = CenteredGrid(0, ZERO_GRADIENT, x=2*Res, y=2*Res, bounds=Box(x=100, y=100))
    inflow   = 0.05 * resample(Sphere(x=source, y=10.0, radius=5), to=smoke, soft=True)
    pressure = None
    
    
    trajectory = [smoke]

    for _ in range(500):
        velocity, smoke, pressure = step(velocity, smoke, pressure)
        trajectory.append(smoke)
        #     plot(vis.overlay(smoke,Walls.geometry), alpha=1, show_color_bar=False, animate='time')
    trajectory = field.stack(trajectory, batch('time'))
    g = hf.create_group(name = 'x-{}'.format(source))
    g.create_dataset('density', data=trajectory.uniform_values().numpy('time,x,y,vector'))
    g.attrs['SourcePosition'] = source

hf.close()

source at  10.0
source at  20.0
source at  30.0
source at  40.0
source at  50.0
source at  60.0
source at  70.0


In [163]:
hf.close()

In [129]:

hf = h5py.File("Smoke-Flow-Data-temp.h5", "w")
hf.create_dataset('density', data=trajectory.uniform_values().numpy('time,x,y,vector'))

<HDF5 dataset "density": shape (31, 400, 400, 1), type "<f4">

In [162]:
hf.keys()

<KeysViewHDF5 ['density 50.0', 'density 80.0']>

In [149]:
hf = h5py.File("Smoke-Flow-Data-temp.h5", "r")

In [154]:
dd = hf['density 50.0']

In [108]:
hf.close()

In [155]:
q = tensor(np.array(dd[:,:,:,0]), batch('time'), spatial('x,y'))
gridQ = CenteredGrid(q, bounds=Box(x=100, y=100))

In [156]:
plot(vis.overlay(gridQ,Walls.geometry), alpha=1,
     show_color_bar=False, animate='time')

In [93]:
p = plot(vis.overlay(gridQ,Walls.geometry), alpha=1,
     show_color_bar=False, animate='time')

In [94]:
p.save('Smoke-x-20.mp4')

CalledProcessError: Command '['ffmpeg', '-f', 'rawvideo', '-vcodec', 'rawvideo', '-s', '1200x500', '-pix_fmt', 'rgba', '-framerate', '10.0', '-loglevel', 'error', '-i', 'pipe:', '-vcodec', 'h264', '-pix_fmt', 'yuv420p', '-y', 'Smoke-x-20.mp4']' returned non-zero exit status 255.

In [88]:
p